In [2]:
pip install --upgrade pip

   ---------------------------------------- 0.0/1.8 MB ? eta -:--:--
   ----- ---------------------------------- 0.3/1.8 MB ? eta -:--:--
   ---------------------- ----------------- 1.0/1.8 MB 3.6 MB/s eta 0:00:01
   ---------------------------------------- 1.8/1.8 MB 4.2 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 25.0.1
    Uninstalling pip-25.0.1:
      Successfully uninstalled pip-25.0.1
Note: you may need to restart the kernel to use updated packages.


In [ ]:
!pip install -q streamlit

In [3]:
!pip install chroma-hnswlib


  Using cached chroma_hnswlib-0.7.6.tar.gz (32 kB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
Failed to build chroma-hnswlib


  error: subprocess-exited-with-error
  
  × Building wheel for chroma-hnswlib (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [5 lines of output]
      running bdist_wheel
      running build
      running build_ext
      building 'hnswlib' extension
      error: Microsoft Visual C++ 14.0 or greater is required. Get it with "Microsoft C++ Build Tools": https://visualstudio.microsoft.com/visual-cpp-build-tools/
      [end of output]
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for chroma-hnswlib
ERROR: ERROR: Failed to build installable wheels for some pyproject.toml based projects (chroma-hnswlib)


  You can safely remove it manually.
  You can safely remove it manually.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pyopenssl 24.2.1 requires cryptography<44,>=41.0.5, but you have cryptography 44.0.3 which is incompatible.
streamlit 1.37.1 requires tenacity<9,>=8.1.0, but you have tenacity 9.1.2 which is incompatible.


In [2]:
!pip install -q crewai openai

  Using cached chroma_hnswlib-0.7.6.tar.gz (32 kB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
Failed to build chroma-hnswlib


  error: subprocess-exited-with-error
  
  × Building wheel for chroma-hnswlib (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [5 lines of output]
      running bdist_wheel
      running build
      running build_ext
      building 'hnswlib' extension
      error: Microsoft Visual C++ 14.0 or greater is required. Get it with "Microsoft C++ Build Tools": https://visualstudio.microsoft.com/visual-cpp-build-tools/
      [end of output]
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for chroma-hnswlib
ERROR: ERROR: Failed to build installable wheels for some pyproject.toml based projects (chroma-hnswlib)


^C


In [1]:
import os
from crewai import Agent, Task, Crew, Process
from langchain.chat_models import ChatOpenAI
import streamlit as st

In [2]:
os.environ["OPENAI_API_KEY"] = "e9217fc4-58e8-4fd4-acb8-b882ae5c2e71"
os.environ["OPENAI_API_BASE"] = "https://api.sambanova.ai/v1"
os.environ["OPENAI_MODEL_NAME"] = "sambanova/Meta-Llama-3.3-70B-Instruct"

In [3]:
custom_llm = ChatOpenAI(
    temperature=0.7,
    model=os.environ["OPENAI_MODEL_NAME"],
    openai_api_base=os.environ["OPENAI_API_BASE"],
    openai_api_key=os.environ["OPENAI_API_KEY"],
)

C:\Users\s\AppData\Local\Temp\ipykernel_23116\3428245507.py:1: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  custom_llm = ChatOpenAI(


In [4]:
filter_agent = Agent(
    role="Prompt Filter",
    goal="Ensure user prompts are appropriate and doesnot contain any insecure words",
    backstory=(
        "You are a strict and ethical filter AI. You detect and reject prompts that contain any form of sexual content, "
        "violence, harassment, sensitive or offensive content, or pedophilia. Your mission is to allow only clean and "
        "safe prompts."
    ),
    verbose=True,
    allow_delegation=False,
    llm=custom_llm
)

In [5]:
def create_filter_task(user_prompt):
    return Task(
        description=(
            f"Review this prompt carefully: \"{user_prompt}\"\n\n"
            "Check if it contains **any** of the following types of content:\n"
            "- Sexual\n"
            "- Sensitive\n"
            "- Harassment or bullying\n"
            "- Violence or threats\n"
            "- Offensive language\n"
            "- Hate speech\n"
            "- Pedophilia\n\n"
            "- If it contains any of the listed inappropriate categories → Respond with 'The prompt is rejected'\n"
            "- Only if it's both clean and safe → Respond with 'Accepted prompt'"
            "The response must be either 'Accepted prompt' or 'The prompt is rejected'"
        ),
        expected_output="Either 'Accepted prompt' or 'The prompt is rejected'",
        agent=filter_agent
    )

In [6]:
def validate_prompt(user_prompt):
    task = create_filter_task(user_prompt)
    crew = Crew(
        agents=[filter_agent],
        tasks=[task],
        process=Process.sequential
    )
    result = crew.kickoff()
    print("\n🧾 Result:", result)
    return result

In [7]:
# Streamlit UI
st.set_page_config(page_title="Prompt Filter", layout="centered")
st.title("🛡️ Prompt Moderation System")

user_prompt = st.text_area("Enter your prompt for moderation:", height=150)

if st.button("Check Prompt"):
    if user_prompt.strip():
        with st.spinner("Validating..."):
            result = validate_prompt(user_prompt)
        st.success("Result:")
        st.markdown(f"**{result}**")
    else:
        st.warning("Please enter a prompt before clicking 'Check Prompt'.")


2025-05-05 15:32:01.744 
  command:

    streamlit run c:\Users\s\anaconda3\Lib\site-packages\ipykernel_launcher.py [ARGUMENTS]
2025-05-05 15:32:01.746 Session state does not function when running a script without `streamlit run`
